In [1]:
# IMPORTAR LIBRERIAS
import sys
import os
import random
import requests
import csv
from bs4 import BeautifulSoup
# from IPython.display import Image
# DRIVER DE SELENIUM
from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options # Para modificar las opciones del webdriver en Chrome.
from selenium.webdriver.common.by import By # Para aceptar las cookies y definir el tipo de búsqueda del elemento.
from selenium.webdriver.common.keys import Keys # Para escribir dentro de una "caja" de búsqueda.
from selenium.webdriver.support.ui import Select
import pandas as pd 
import time
import json
import sqlite4
from fpdf import FPDF

In [2]:
# PASOS OBLIGATORIOS PARA INICIALIZAR EL WEBDRIVER DE CHROME PARA REALIZAR EL WEBSCRAPING, QUE SIGUE FRANK ANDRADE
#path = '/Users/Eduardo/......'
#service = Service (executable_path=path)
#driver = webdriver.Chrome(service=service) 
#driver.get(web) . PARA TESTEAR SI ESTA FUNCIONANDO BIEN o NO, LA INICIACION DEL WEBDRIVER.


# CREO UNA LISTA CON TODOS LOS AÑOS DE LOS MUNDIALES DE FUTBOL CELEBRADOS.
año_mundial = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966,
                1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998,
                2002, 2006, 2010, 2014, 2018, 2022]

driver = webdriver.Chrome()
web = 'https://en.wikipedia.org/wiki/1982_FIFA_World_Cup'
driver.get(web)

# EXAMINANDO LA WEB.
# PARA ENCONTRAR LOS PARTIDOS DE FUTBOL, USANDO xpath.
partidos_futbol = driver.find_elements(by='xpath',value='//div[@class="footballbox"]/table[@class="fevent"]')


# CREO LISTAS PARA CADA UNO DE LOS DATOS BUSCADOS (local, marcador y visitante).
local = []
marcador = []
visitante = []
# CREO UN BUCLE FOR PARA RECORRER CADA PARTIDO DE FUTBOL y PODER OBTENER LOS DATOS QUE QUIERO (local, marcador y visitante). 
for partido in partidos_futbol:
  local.append(partido.find_element(by='xpath',value='.//th[1]').text)
  marcador.append(partido.find_element(by='xpath',value='.//th[2]').text)
  visitante.append(partido.find_element(by='xpath',value='.//th[3]').text) 


# GENERO UN DATAFRAME BASADO EN UN DICCIONARIO (TAMBIEN puedo hacerlo sobre LISTAS).
dict_futbol = {"Local": local, "Marcador": marcador, "Visitante": visitante}
df_futbol = pd.DataFrame(dict_futbol)

# CREO UNA COLUMNA ("Año") DENTRO DEL DATAFRAME.
df_futbol["Año"] = 1982

#GENERO UNA ESPERA PARA QUE EL WEBSCRAPING SEA EFICAZ.
time.sleep(3)

# EXPORTO EL DATAFRAME (df_futbol) A UN ARCHIVO CSV.
df_futbol.to_csv("Mundial_1982.csv")

# CIERRO EL DRIVER.
driver.quit()

In [3]:

driver = webdriver.Chrome()

# CREO UNA FUNCION PARA OBTENER LA INFO QUE BUSCO, EN TODAS LAS WEBS DE LOS DIFERENTES AÑOS DE LOS MUNDIALES DE FUTBOL.
def obtener_datos(año_mundial):
  web = f'https://en.wikipedia.org/wiki/{año_mundial}_FIFA_World_Cup' 
  # LAS LLAVES de la VARIABLE ({año_mundial}) ME PERMITEN HACER QUE LA BÚSQUEDA SEA DINAMICA y 
  # NO SE LIMITE TAN SOLO AL AÑO QUE YO LE ESCRIBA.

  driver.get(web)

  #Examinando la web.
  partidos_futbol = driver.find_elements(by='xpath',value='//div[@class="footballbox"]/table[@class="fevent"]')

  local = []
  marcador = []
  visitante = []

  for partido in partidos_futbol:
    local.append(partido.find_element(by='xpath',value='.//th[1]').text)
    marcador.append(partido.find_element(by='xpath',value='.//th[2]').text)
    visitante.append(partido.find_element(by='xpath',value='.//th[3]').text) 

  dict_futbol = {"Local": local, "Marcador": marcador, "Visitante": visitante}
  df_futbol = pd.DataFrame(dict_futbol)
  df_futbol["Año"] = año_mundial
  time.sleep(3)
  return df_futbol # DEVUELVE EL DATAFRAME


año_mundial = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 
        1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998, 
        2002, 2006, 2010, 2014, 2018, 2022]

# CON ESTA LISTA DE COMPRENSION, GENERO UN BUCLE FOR y GUARDO LOS RESULTADOS DE TODAS LAS LISTAS
# OBTENIDAS, EN UNA SOLA LISTA.
fifa_mundiales = [obtener_datos(año) for año in año_mundial] # Esto es una lista de comprension.


# HAGO UN BUCLE FOR PARA ITERAR = RECORRER. A CADA AÑO, APLICAR LA FUNCION CREADA PERVIAMENTE (obtener_datos).
#for año in año_mundial:
  #obtener_datos(año)
# ESTE BUCLE FOR, LE QUITARÉ PARA CREAR LA LISTA DE COMPRENSION QUE HE PUESTO JUSTO ENCIMA y,
# QUE RECIBE EL NOMBRE DE (fifa_mundiales).

driver.quit()

# JUNTO o CONCATENO TODO LO QUE HAY DENTRO DE LA LISTA fifa_mundiales (los 20 dataframes).
df_fifa_mundiales = pd.concat(fifa_mundiales, ignore_index=True)

# EXPORTO EL DATAFRAME GENERADO.
df_fifa_mundiales.to_csv("FIFA Mundiales futbol.csv", index=False)